In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import csv

In [4]:
#PULLING SLUG INFO THAT WILL LEAD TO THE PRODUCT PAGE

#credit for this code: https://www.hackerearth.com/fr/practice/notes/praveen97uma/crawling-a-website-that-loads-content-using-javascript-with-selenium-webdriver-in-python/
#this code creates a function that gets the browser to scroll down
def scrollDown(driver, n_scroll):
    body = driver.find_element_by_tag_name("body")
    while n_scroll >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        n_scroll -= 1
    return driver

#chromedriver needs to be downloaded from https://chromedriver.chromium.org/downloads
chrome_path = "/Users/yelenanevel/Downloads/chromedriver"
#opens the google chrome
driver = webdriver.Chrome(executable_path = chrome_path)

#this list represents different skin care categories to collect
categories = ['all-cleansers',
             'all-moisturizers',
             'masks-1',
             'treatments',
             'eye-care',
             'sunscreen',
             'lip-care']

#creating an empty data frame that all the URLs and categories will go in to
slugs_df = pd.DataFrame(columns = ['category', 'href'])

#looping through all the categories
for category in categories:
    #the url that leads to the list of products
    url = "https://credobeauty.com/collections/" + category
    driver.get(url)
    time.sleep(20)

    old_len = 0
    while True:
        browser = scrollDown(driver, 20) #scroll down the page
        time.sleep(10) #give it time to load
        slug = driver.find_elements_by_class_name("product") #look for the available info on page
        new_len = len(slug)
        if old_len == new_len: #if the old length and new length are equal, the bottom of page was reached
            break
        else:
            old_len = new_len
    #this gets all the page_soure
    html = driver.page_source
    #converts it to soup so we can extract info
    soup = BeautifulSoup(html)
    #finding all the products
    page = soup.find_all('div', {'class':"product"})

    slugs = []
    for product in page:
        slug = {}
        slug['category'] = category
        #finding URL under the class: product div
        slug['href'] = product.find('a')['href'] #pulls out the slug from the page source
        #appending the list with dictionary
        slugs.append(slug)
    #making a dataframe from list
    df = pd.DataFrame(slugs)
    #appending this to final dataframe with all the URLs
    slugs_df = pd.concat([slugs_df, df], axis = 0, ignore_index = True, sort = True)
    
#closing the chrome window    
driver.close()

#saving the final dataframe of URLs in to a csv folder
slugs_df.to_csv('../data/credo_product_slugs.csv', index = False)

In [5]:
slugs_df.head()

,category,href
0,all-cleansers,/collections/all-cleansers/products/one-love-o...
1,all-cleansers,/collections/all-cleansers/products/exfoliatin...
2,all-cleansers,/collections/all-cleansers/products/josh-roseb...
3,all-cleansers,/collections/all-cleansers/products/indie-lee-...
4,all-cleansers,/collections/all-cleansers/products/indie-lee-...


In [6]:
#GETTING HTML FILES THAT WILL LEAD TO THE PRODUCT PAGE

for i in slugs_df.index: #for every url slug we got
    url = "https://credobeauty.com" + slugs_df.loc[i, 'href']
    res = requests.get(url)
    time.sleep(20)
    if res.status_code != 200:
        print('error when requestion {}'.format(url))
        break
    soup = BeautifulSoup(res.content, 'html.parser')
    #going to save it to this path
    path = './soups/credosoup' + str(i) + '.html'
    with open(path, 'w') as f:
        f.write(str(soup))

In [7]:
#GATHER INFO FROM THE PRODUCT HTML PAGES

#opening a new csv file with headers
header = ['name', 'brand', 'category', 'price', 'ingredients']

with open('../data/credo_product_info.csv', "w", newline='') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(header) # write the header
    

for i in slugs_df.index:
    products = [] #empty list to append the dictionary in to before passing in to a DataFrame
    #setting the path to the the html files
    path = './soups/credosoup' + str(i) + '.html'
    #opening and reading the file
    file_path = open(path, 'rb')
    file_read = file_path.read()
    #creating a new soup from it
    soup = BeautifulSoup(file_read, 'html.parser')
    #gathering the data from the pages
      
    try:          
        product = {}
        product['name'] = str(soup.find('h1', {'class': 'product_name'}).text).split('\n')[2].strip()
        product['brand'] = soup.find('span', {'class': 'vendor'}).text
        product['category'] = slugs_df.category[i]
        product['price'] = str(soup.find('span', {'class': 'current_price'}).text).split(' ')[1].strip()
        product['ingredients'] = soup.find('div', {'class': 'tab tab3 ingredients'}).text


            #append the empty list to later make in to a dataframe
        products.append(product)
        product_df = pd.DataFrame(products) 
            #append in the csv file that created above
        product_df.to_csv('../data/credo_product_info.csv', mode='a', index = False, header = False)
    except:
        print(i)
    
   

50
77
161
204
253
273
611


In [8]:
#i = 50 is a Indie Lee Clarity Kit
# i = 77 is a Skin Brush
# i = 161 is a The New Normal Kit
# i = 204 Indie Lee Clarity Kit
# i = 253 is a The "Clean 5" Starter Kit

#all of these products can be ommitted since they are either already represented individually
#or are not a skincare liquid product